In [51]:
import json
import numpy as np

In [54]:
i = 0
with open('./ElectricLoadServingEntities_IOU_POU/footprint_sdiego.geojson', 'r') as f:
    data = json.load(f)
    heights_by_building_type = {
        j: np.nanmean([
            float(''.join([m for m in k['properties'].get('height', '') if m not in ['m', ' ']]).split(';')[0])
            if k['properties'].get('building') == j and k['properties'].get('height') not in [None, '']
            else np.nan
            for k in data.get('features', [])
        ])
        for j in set(k['properties'].get('building') for k in data.get('features', []) if k['properties'].get('building'))
    }
    mean_height = np.nanmean(list(heights_by_building_type.values()))
    heights_by_building_type = {
        k: (v if not np.isnan(v) else mean_height)
        for k, v in heights_by_building_type.items()
    }
    for feature in data.get('features'):
        # print(np.mean(feature.get('geometry').get('coordinates')[0][0], axis=0))
        h=0
        if feature.get('properties').get('height') is not None:
            # print(feature.get('properties').get('height'))
            h = float(''.join([i if i not in ['m', ' '] else '' for i in feature.get('properties').get('height')]).split(';')[0] )
        if feature.get('properties').get('height_ft') is not None:
            # print(feature.get('properties').get('height_ft'))
            h = float(''.join([i if i not in ['f','t',"'",'"', ' '] else '' for i in feature.get('properties').get('height_ft')])) * 0.3048
        else:
            h = heights_by_building_type.get(feature.get('properties').get('building'), mean_height)
        feature.get('properties')['height'] = np.round(h,decimals=2)


/tmp/ipykernel_831824/2354251039.py:5: RuntimeWarning: Mean of empty slice
  j: np.nanmean([


In [55]:
with open('./ElectricLoadServingEntities_IOU_POU/footprint_sdiego_with_heights.geojson', 'w') as f:
    json.dump(data, f)  
        